In [1]:
#files containing fusion events folder
path='/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/'
files = ['Prm1_ctrl'] 

Myo= ['Myo52_M', 'Myo52_P']
RefM= '_B' #'_B' if POI in h+ cell, '_R' if POI in h- cell
RefP= '_R' #'_R' if POI in h+ cell, '_B' if POI in h- cell

In [2]:
from trajalign.traj import Traj
from trajalign.average import load_directory
import numpy as np
from matplotlib import pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import norm as norm


In [3]:
def replicats_nb(POI):
    rep_list = [ folder for folder in os.listdir(path+POI+'/aligned_trajectories/') if 'Rep_' in folder ]
    r= len(rep_list)
    return r

def get_data(POI,rep) :
    directory_rep = os.listdir( path+POI+'/aligned_trajectories/Rep_'+str(rep+1)+'/') #absolute path of directory to charge data from
    f = [ folder for folder in directory_rep if 'cell_' in folder ] # 'cell_' -> folder nam to list
    return f 

n= len(files) 
Rep= []
for i in range(n):
    r=replicats_nb(files[i])
    Rep.append(r)


folders=[[] for _ in range(n)] #lists for POI rep folders => Subsets: POI then Rep
Cells=[[] for _ in range(n)] #lists of trajectories for each events for each POI of each replicate

for i in range(len(Rep)):
    folders[i]=[[] for _ in range(Rep[i])] #prep placeholder for directory of each replicats
    Cells[i]=[[] for _ in range(Rep[i])] 

# list all cell_ folders for each POI 

for i in range(n) :
    for r in range(Rep[i]):
        f=get_data(files[i],r)
        folders[i][r]= f 

In [4]:
ct=0
for f in folders:
    r=1
    i=0 
    for rep in f:
        for event in rep: 
            t_l = load_directory(path+files[ct] +'/aligned_trajectories/Rep_'+str(r)+'/'+ event ,pattern= 'Traj_', comment_char = '#', t_unit='min' , frames = 0 ,t= 1, coord = ( 2 , 3 ) , f = 4 ,coord_unit = 'pxl' )
            Cells[ct][i].extend(t_l)
        r+=1
        i+=1
    ct+=1
    

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the trajectories is 'None' <<

Traj_M.txt
Traj_P.txt
Traj_POI.txt

 >> load_directory: The 'intensity_normalisation' applied to the tra

In [5]:
# prime lists
POI=[[] for _ in range(n)]
Myo52= [[] for _ in range(2)] #get Myo52 data in one list; [0] = Myo_M, [1] = Myo_P 
Myo_P=[[] for _ in range(n)]
Myo_M=[[] for _ in range(n)]
for i in range(len(Rep)):
    POI[i]=[[] for _ in range(Rep[i])] 
    Myo_P[i]=[[] for _ in range(Rep[i])] 
    Myo_M[i]=[[] for _ in range(Rep[i])]
    
# sort trajectories according to origin 

for i in range(n):
        for r in range (len(Cells[i])):
            for t in Cells[i][r]:
                if RefP in t.annotations() ['file'] : 
                    Myo_P[i][r].append(t)
                    Myo52[1].append(t)
                    
                elif RefM in t.annotations() ['file'] :
                    Myo_M[i][r].append(t)
                    Myo52[0].append(t) 
                else:
                    POI[i][r].append(t)
                    

In [6]:
def directed_distance (list_1, list_2,POI,rep, what='P_POI'):  # dist from list1 -> list2
    Dist=[]
    for i in range (len(list_1)):
        d_1= list_1[i]
        d_2= list_2 [i]
        inter=set(d_1.frames()).intersection(d_2.frames())  # get list of common frames between the two trajs
        U=np.empty( (len(inter),2))      
        dist=[] 
        frame= []       
        for it in inter:
            it= int(it)
            frame.append(it)
        frame.sort()
        if len(frame)>0:
            
            for j in frame: 
            # get coordinates and calculate the distance between the trajectories for each shared frame. 
                for f in range (len(d_1)):         
                    if d_1.frames(f) == j:
                        x_1= d_1.coord(f) [0]
                for f in range (len(d_2)):
                    if d_2.frames(f) == j:
                        x_2= d_2.coord(f) [0]
                d = (x_1-x_2)*1000
                dist.append(d)              # create list of dist
            for j in range(len(inter)): 
                U[j,0]= frame[j]
                U[j,1] = dist [j]                    # create matrice with shared frames in [:,0] and distances in [:,1]
            d= Traj()
            d.input_values('frames', U[:,0])
            d.input_values('f', U[:,1])
            d.fill()    
            d.time(3,min) 
            Dist.append(d)
            d.save(path+POI+'/aligned_trajectories/Rep_'+str(rep)+'/cell_%03d/dir_dist_%s.txt'%(i, what))
            print (path+POI+'/aligned_trajectories/Rep_'+str(rep)+'/cell_%03d/dir_dist_%s.txt'%(i, what))
        elif len(frame)==0 :
            d= Traj()
            Dist.append(d)
            d.save(path+POI+'/aligned_trajectories/Rep_'+str(rep)+'/cell_%03d/dir_dist_%s.txt'%(i, what))
            print (path+POI+'/aligned_trajectories/Rep_'+str(rep)+'/cell_%03d/dir_dist_%s.txt'%(i, what))
            print('no traj overlap')
        
        
    return Dist     

In [7]:
# prime list for distances
vect_d_P_POI= [[] for _ in range(n)] # directed distance from Myo52_P towards POI for each events
vect_d_P_M= [[] for _ in range(n)]   # directed distance from Myo52_P towards Myo52_M for each events
vect_d_M_POI= [[] for _ in range(n)]  #  directed distance from Myo52_M towards POI for each events 
for i in range(len(Rep)):
    vect_d_P_POI[i]=[[] for _ in range(Rep[i])] 
    vect_d_M_POI[i]=[[] for _ in range(Rep[i])] 
    vect_d_P_M[i]= [[] for _ in range(Rep[i])]
    


for i in range(len(Myo_P)):
    for r in range (len (Myo_P[i])):
        rep= r+1
        vect_d_P_POI[i][r]= directed_distance(POI[i][r], Myo_P[i][r],files[i],rep, 'P_POI') #distance from Myo52 -> POI
        vect_d_P_M[i][r]= directed_distance(Myo_M[i][r], Myo_P[i][r], files[i],rep,'P_M') # Myo_P -> Myo_M 
        vect_d_M_POI[i][r]= directed_distance(Myo_M[i][r], POI[i][r],files[i],rep, 'M_POI') #distance POI -> Myo_M

/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_000/dir_dist_P_POI.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_001/dir_dist_P_POI.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_002/dir_dist_P_POI.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_003/dir_dist_P_POI.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_004/dir_dist_P_POI.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_005/dir_dist_P_POI.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_006/dir_dist_P_POI.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_007/dir_dist_P_POI.txt


/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_047/dir_dist_P_M.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_048/dir_dist_P_M.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_049/dir_dist_P_M.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_050/dir_dist_P_M.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_051/dir_dist_P_M.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_052/dir_dist_P_M.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_053/dir_dist_P_M.txt
/mnt/c/Users/Valentine/Documents/Analysis/POI_spread_signal/Prm1_ctrl/aligned_trajectories/Rep_1/cell_054/dir_dist_P_M.txt
/mnt/c/Users/Val

In [8]:
av_POI= [[] for _ in range(n)]
av_Myo_P= [[] for _ in range(n)]
av_Myo_M= [[] for _ in range(n)]
hist_POI=[[]for _ in range (n)]
hist_Myo_P= [[] for _ in range(n)]
hist_Myo_M= [[] for _ in range(n)]
Disp_POI=[[] for _ in range(n)]
Disp_Myo_P= [[] for _ in range(n)]
Disp_Myo_M= [[] for _ in range(n)]
for k in range(len(Rep)):
    av_POI[k]= [[] for _ in range(Rep[k])]
    av_Myo_P[k]= [[] for _ in range(Rep[k])]
    av_Myo_M[k]= [[] for _ in range(Rep[k])]
    hist_POI[k]= [[] for _ in range(Rep[k])]
    hist_Myo_P[k]= [[] for _ in range(Rep[k])]
    hist_Myo_M[k]= [[] for _ in range(Rep[k])]
    Disp_POI[k]= [[] for _ in range(Rep[k])]
    Disp_Myo_P[k]= [[] for _ in range(Rep[k])]
    Disp_Myo_M[k]= [[] for _ in range(Rep[k])]
    
    
#prime list for median informations of each POI
tot_POI= [[] for _ in range(n)]
tot_Myo_P= [[] for _ in range(n)]
tot_Myo_M= [[] for _ in range(n)]
tot_hist_POI=[[]for _ in range (n)]
tot_hist_Myo_P= [[] for _ in range(n)]
tot_hist_Myo_M= [[] for _ in range(n)]
tot_Disp_POI=[[] for _ in range(n)]
tot_Disp_Myo_P= [[] for _ in range(n)]
tot_Disp_Myo_M= [[] for _ in range(n)]

In [9]:
## functions to estimate standard deviation from Median average deviation
def mad (s):
    list_s=[]
    m=np.median(s)
    for x in s: 
        s_i=np.absolute(x-m)
        list_s.append(s_i)
   
    mad_s= np.median(list_s)
    mad=mad_s*1.4826
    return mad

def mad_ln( s ) :
    sigma_ln = mad( np.log( s ) )
    m = np.median( s )
    sigma = m * sigma_ln
    return sigma

# function for position and fluorescence average

# return histograms of data, U = average data info , D= dispersion 

def average (list, POI,nR, what='POI',sample='rep') :
    print(POI)
    f_min =0 
    f_max= 0
    if sample=='tot':
        list=[item for sublist in list for item in sublist]
    for t in list: 
        if len(t)>0:
            if ( (min(t.frames()))< f_min) :
                f_min = min(t.frames())
                
            if ( (max(t.frames())) >f_max) :
                f_max = max(t.frames()) 
    
    row= f_max-f_min
    U= np.empty((row,7))   # column 0: frame,1: x, 2: y, 3: f, 4: err_x, 5: err_y, 6: err_f
    D= np.empty((row,2))
    hist_f=[[] for _ in range(row)]
    hist_x=[[] for _ in range(row)]
    hist_y=[[] for _ in range(row)]
    hist=[hist_x,hist_y,hist_f]
    i =0
    for frame in range (f_min,f_max):   #range of frame for which to average fluorescence
        fluo=[]
        x=[]
        y=[]
        time=[]
        for t in list :
            if len(t)>0:
                for ct in range(len(t)): 
                
                    if t.frames(ct)== frame:
                        fluo.append(t.f(ct))
                        x.append(t.coord(ct)[0])
                        y.append(t.coord(ct)[1]) 
                        time.append(t.frames(ct))
       
        time=np.array(time)             
        U[i,0]= np.median(time)
        D[i,0] =np.median(time)
        fluo=np.array(fluo)
        fluo= fluo[~np.isnan(fluo)]
        x=np.array(x)
        x = x[~np.isnan(x)]
        y=np.array(y)
        y = y[~np.isnan(y)]
        
        if (fluo.size>0):
            U[i,3]= np.median(fluo)
            mad_f = mad_ln(fluo)
            U[i,6]= mad_f 
            hist_f[i]= fluo 
         
        if (x.size>0):
            U[i,1]= np.median(x)
            mad_x= mad(x)
            U[i,4]=mad_x
            hist_x[i]= x
            
        if (y.size>0):
            U[i,2]= np.median(y)
            mad_y = mad(y)
            U[i,5]= mad_y 
            hist_y[i]= y 
            dist=[]
        if (y.size>0) and (x.size>0): 
            D[i,1]= (np.sqrt(mad_x**2+mad_y**2))    
        i+=1
    av=Traj()
    av.input_values('frames',U[:,0])
    av.input_values('coord', (U[:,1], U[:,2]))
    av.input_values('coord_err', (U[:,4], U[:,5]))
    av.input_values('f',U[:,3])   
    av.input_values('f_err',U[:,6])
    av.fill() 
    av.time(3,min) 
    disp=Traj()
    disp.input_values('frames',D[:,0])
    disp.input_values('f',D[:,1])
    disp.time(3,min) 
    if sample=='rep':
        os.makedirs(path+POI+'/aligned_trajectories/Data/average/', exist_ok=True)
        av.save(path+POI+'/aligned_trajectories/Data/average/Average_coordinates_%s_Rep_%s.txt'%(what,str(nR+1)))
        disp.save(path+POI+'//aligned_trajectories/Data/average/Dispersion_%s_Rep_%s.txt'%(what,str(nR+1)))
    if sample=='tot':
        os.makedirs(path+POI+'/aligned_trajectories/Data/average/', exist_ok=True)
        av.save(path+POI+'/aligned_trajectories/Data/average/Average_coordinates_%s.txt'%what)
        disp.save(path+POI+'/aligned_trajectories/Data/average/Dispersion_%s.txt'%what)
        
            
       
    return av, hist, disp


In [10]:
for i in range(len(POI)):
    for r in range (len(POI[i])):
        av_POI[i][r], hist_POI[i][r], Disp_POI[i][r]= average(POI[i][r],files[i],r, what= 'POI',sample='rep')
        av_Myo_M[i][r], hist_Myo_M[i][r], Disp_Myo_M[i][r]= average(Myo_M[i][r], files[i],r, what='Myo_M',sample='rep')
        av_Myo_P[i][r], hist_Myo_P[i][r], Disp_Myo_P[i][r]= average(Myo_P[i][r], files[i],r, what='Myo_P',sample='rep')
        
for i in range(len(POI)):
    tot_POI[i],tot_hist_POI[i], tot_Disp_POI[i]= average(POI[i],files[i],0, what= 'POI',sample='tot')
    tot_Myo_M[i], tot_hist_Myo_M[i], tot_Disp_Myo_M[i]= average(Myo_M[i],files[i],0, what= 'MyoM',sample='tot')
    tot_Myo_P[i], tot_hist_Myo_P[i], tot_Disp_Myo_P[i]= average(Myo_P[i],files[i],0, what= 'MyoP',sample='tot')


Prm1_ctrl
Prm1_ctrl


/tmp/ipykernel_2006/1778943478.py:14: RuntimeWarning: divide by zero encountered in log
  sigma_ln = mad( np.log( s ) )
/tmp/ipykernel_2006/1778943478.py:6: RuntimeWarning: invalid value encountered in double_scalars
  s_i=np.absolute(x-m)


Prm1_ctrl
Prm1_ctrl
Prm1_ctrl
Prm1_ctrl


In [11]:
def av_distance (list, POI,nR, what= 'P_POI',sample='rep'):
#get average  
    print(POI)
    if sample=='tot':
        list=[item for sublist in list for item in sublist]
    f_min =0 
    f_max= 0
    for t in list: 
        if len(t)>0:
            if ( (min(t.frames())< f_min)) :
                f_min = int(min(t.frames()))
            if ( (max(t.frames())) >f_max) :
                f_max = int(max(t.frames()) )
    row= f_max-f_min
    D= np.empty((row,3))
    hist_dist=[[] for _ in range(row)]
    index = 0
    for frame in range( f_min, f_max):
        distance=[]
        time=[]
        for t in list :
            if len(t)>0:
                for ct in range (len(t)):
                    if t.frames(ct) == frame:
                        time.append(t.t(ct))
                        distance.append(t.f(ct))
        time=np.array(time)            
        D[index,0] =np.median(time)
        distance=np.array(distance)
        distance = distance[~np.isnan(distance)]          
        if (distance.size>0):        
            D[index,1]= np.median(distance) 
            D[index,2] = mad(distance)
            hist_dist[index]=distance
        index+=1
    av=Traj()
    av.input_values('t',D[:,0])
    av.input_values('f',D[:,1])
    av.input_values('f_err',D[:,2])
    av.fill() 
    if sample=='rep':
        os.makedirs(path+POI+'/aligned_trajectories/Data/average/', exist_ok=True)
        av.save(path+POI+'/aligned_trajectories/Data/average/Average_%s_Rep_%s.txt'%(what,str(nR+1)))
    if sample=='tot':
        os.makedirs(path+POI+'/aligned_trajectories/Data/average/', exist_ok=True)
        av.save(path+POI+'/aligned_trajectories/Data/average/Average_%s.txt'%what)
          
    return av, hist_dist


In [12]:
##prime lists

av_dir_P_POI= [[] for _ in range(n)]
av_dir_P_M=[[] for _ in range(n)]
av_dir_M_POI= [[] for _ in range(n)] # av distance between Myo52_M-POI
hist_distP_POI=[[] for _ in range(n)]
hist_distP_M= [[] for _ in range(n)]
hist_distM_POI= [[] for _ in range(n)]
for k in range(len(Rep)):
    av_dir_P_POI[k]= [[] for _ in range(Rep[k])]
    av_dir_P_M[k]= [[] for _ in range(Rep[k])]
    av_dir_M_POI[k]= [[] for _ in range(Rep[k])]
    hist_distP_POI[k]= [[] for _ in range(Rep[k])]
    hist_distP_M[k]= [[] for _ in range(Rep[k])]
    hist_distM_POI[k]= [[] for _ in range(Rep[k])]

for i in range(len(POI)):
    for r in range (len(POI[i])):
        av_dir_P_POI[i][r], hist_distP_POI[i][r]=av_distance(vect_d_P_POI[i][r], files[i],r,what='dist_P_POI',sample='rep')
        av_dir_P_M[i][r], hist_distP_M[i][r]=av_distance(vect_d_P_M[i][r], files[i],r,what='dist_P_M',sample='rep')
        av_dir_M_POI[i][r], hist_distM_POI[i]=av_distance(vect_d_M_POI[i][r], files[i],r,what='dist_M_POI',sample='rep')

Prm1_ctrl
Prm1_ctrl
Prm1_ctrl


In [13]:
tot_av_dir_P_POI= [[] for _ in range(n)]
tot_av_dir_P_M=[[] for _ in range(n)]
tot_av_dir_M_POI= [[] for _ in range(n)] # av distance between Myo52_M-POI
tot_hist_distP_POI=[[] for _ in range(n)]
tot_hist_distP_M= [[] for _ in range(n)]
tot_hist_distM_POI= [[] for _ in range(n)]

for i in range(len(POI)):
    tot_av_dir_P_POI[i], tot_hist_distP_POI[i]=av_distance(vect_d_P_POI[i], files[i],0,what='dist_P_POI',sample='tot')
    tot_av_dir_P_M[i], tot_hist_distP_M[i]=av_distance(vect_d_P_M[i], files[i],0,what='dist_P_M',sample='tot')
    tot_av_dir_M_POI[i], tot_hist_distM_POI[i]=av_distance(vect_d_M_POI[i], files[i],0,what='dist_M_POI',sample='tot')

Prm1_ctrl
Prm1_ctrl
Prm1_ctrl
